In [1]:
import math
import numpy as np
from collections import namedtuple
from decimal import *

from typing import List, Tuple
from enum import Enum

import xlrd

# 変更履歴

- 2019.10.15: 浴室の床が断熱、浴室の床が外皮に接しない場合の計算を追加  
それにより、外皮の部位の面積の合計が異なることが判明した。（2019.10時点の評価方法においては、床断熱住戸については同じ面積を使用している。）

# 四捨五入

建研HPで公表する値としては、さらに小数点2桁までにまとめた値とする。  
丸める場合に、round関数は、JIS Z 8401 付属書A の方法（0.5の場合は偶数側に丸める）が適用されるため、自前に作成したround2関数で丸めることとする。この場合は、端数が「.5」の場合は、必ず大きい側（切り上げ側）に丸められる。

In [2]:
def round2(x,d=2):
    p=10**d
    return (x*p*2+1)//2/p
#    return x.quantize(Decimal('0.01'), rounding=ROUND_HALF_UP)

In [3]:
def round2_direction(x, d=2):
    return (
        round2(x[0], d),
        round2(x[1], d),
        round2(x[2], d),
        round2(x[3], d)
    )

# 住宅の種類

床・基礎部分の断熱方法

In [4]:
class FloorIns(Enum):

    # 床断熱
    FLOOR = 1
    
    # 基礎断熱
    BASE = 2

浴室における床・基礎部分の断熱方法

In [5]:
class BathIns(Enum):

    # （浴室が2階にある等）浴室床が外皮に面さない
    NOT_EXIST = 1
    
    # 床断熱
    FLOOR = 2
    
    # 基礎断熱
    BASE = 3

# 保持するパラメータ

In [6]:
class House():
    
    def __init__(self, name, ver, floor_ins, bath_ins):
        
        # 名前
        self.name = name
        
        # バージョン
        self.ver = ver

        # 床の断熱種類
        self.floor_ins = floor_ins
        
        # 浴室の断熱種類
        self.bath_ins = bath_ins
        
        # 床面積の合計, m2
        self.a_f_total = None
        
        # 形状係数
        self.sf = None
        
        # 1階の床面積に対する2階の床面積の比
        self.r_fa = None
        
        # 基礎の高さ（一般部分）, m
        self.h_base = None
        
        # 基礎の高さ（玄関）, m
        self.h_base_entrance = None
        
        # 基礎の高さ（浴室）, m
        self.h_base_bath = None
        
        # 高さ, m
        self.h = None
        
        # ドアの面積（玄関）, m2
        self.a_door_entrance = None
        
        # ドアの面積（勝手口）, m2
        self.a_door_backentrance = None
        
        # 玄関の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_entrance_ms = None
        
        # 玄関の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_entrance_os = None
        
        # 浴室の土間床等の周辺部の基礎長さ（主開口方位側）, m
        self.l_base_bath_ms = None
        
        # 浴室の土間床等の周辺部の基礎長さ（その他方位側）, m
        self.l_base_bath_os = None
        
        # 開口部の面積比率
        self.r_opn = None
        
        # 方位別窓面積比率
        self.r_wnd = None
        
        # 床面積（土間床含む）, m2        
        self.a_f = None
        
        # 周長, m
        self.l_prm = None

        # 長辺の長さ, m
        self.l_ls = None
        
        # 短辺の長さ, m
        self.l_ss = None
        
        # 玄関の土間床等の面積, m2
        self.a_ef_entrance = None
        
        # 浴室の土間床等の面積, m2
        self.a_ef_bath = None
        
        # その他の土間床等の面積, m2
        self.a_ef_other = None
        
        # 土間床等の面積の合計, m2
        self.a_ef_total = None
        
        # 玄関等の土間床等の外周部の長さ（外気側）, m
        self.l_base_entrance_outside = None
        
        # 玄関等の土間床等の外周部の長さ（床下側）, m
        self.l_base_entrance_inside = None
        
        # 浴室の土間床等の外周部の長さ（外気側）, m
        self.l_base_bath_outside = None
        
        # 浴室の土間床等の外周部の長さ（床下側）, m
        self.l_base_bath_outside = None
        
        # その他の土間床等の外周部の長さ（外気側）, m
        self.l_base_other_outside = None
        
        # その他の土間床等の外周部の長さ（床下側）, m
        self.l_base_other_inside = None
        
        # 土間床周長（外気側）, m
        self.l_prm_outside = None
        
        # 土間床周長（床下側）, m
        self.l_prm_inside = None
        
        # 浴室の床断熱した床の面積, m2
        self.a_if_bath = None
        
        # 床断熱した床の面積, m2
        self.a_if = None
        
        # 屋根又は天井の面積, m2
        self.a_roof = None
        
        # 玄関等の基礎の面積（外気側）, m2
        self.a_base_entrance_outside = None
        
        # 玄関等の基礎の面積（床下側）, m2
        self.a_base_entrance_inside = None
        
        # 浴室の基礎の面積（外気側）, m2
        self.a_base_bath_outside = None
        
        # 浴室の基礎の面積（床下側）, m2
        self.a_base_bath_inside = None
        
        # その他の基礎の面積（外気側）, m2
        self.a_base_other_outside = None
        
        # その他の基礎の面積（床下側）, m2
        self.a_base_other_inside = None
        
        # 基礎の面積の合計（外気側）, m2
        self.a_base_total_outside = None
        
        # 基礎の面積の合計（床下側）, m2
        self.a_base_total_inside = None
        
        # 垂直外皮面積, m2
        self.a_srf = None
        
        # 基礎を除く外皮の部位の面積の合計, m2
        self.a_dash_env = None
        
        # 外皮の部位の面積の合計, m2
        self.a_env = None
        
        # 開口部の面積の合計, m2
        self.a_opn = None
        
        # 窓の面積の合計, m2
        self.a_wnd_total = None

        # 窓の面積, m2
        self.a_wnd = None
        
        # ドアの面積, m2
        self.a_door = None

        # 壁の面積, m2
        self.a_wall = None
        
        # rnd_床面積の合計, m2
        self.rnd_a_f_total = None
        
        # rnd_外皮の部位の面積の合計, m2
        self.rnd_a_env = None

        # rnd_屋根又は天井の面積, m2
        self.rnd_a_roof = None
        
        # rnd_壁の面積, m2
        self.rnd_a_wall = None

        # rnd_窓の面積, m2
        self.rnd_a_wnd = None

        # rnd_ドアの面積, m2
        self.rnd_a_door = None

        # rnd_浴室の床断熱した床の面積, m2
        self.rnd_a_if_bath = None

        # rnd_床断熱した床の面積, m2
        self.rnd_a_if = None

        # rnd_土間床等の面積の合計, m2
        self.rnd_a_ef_total = None

        # rnd_浴室の土間床等の面積, m2
        self.rnd_a_ef_bath = None

        # rnd_基礎の面積の合計（外気側）, m2
        self.rnd_a_base_total_outside = None

        # rnd_基礎の面積の合計（床下側）, m2
        self.rnd_a_base_total_inside = None

        # rnd_玄関等の基礎の面積（外気側）, m2
        self.rnd_a_base_entrance_outside = None

        # rnd_玄関等の基礎の面積（床下側）, m2
        self.rnd_a_base_entrance_inside = None

        # rnd_浴室の基礎の面積（外気側）, m2
        self.rnd_a_base_bath_outside = None

        # rnd_浴室の基礎の面積（床下側）, m2
        self.rnd_a_base_bath_inside = None

        # rnd_玄関等を除く基礎の面積（外気側）, m2
        self.rnd_a_base_except_entrance_outside = None

        # rnd_玄関等を除く基礎の面積（床下側）, m2
        self.rnd_a_base_except_entrance_inside = None

        # rnd_その他の基礎の面積（外気側）, m2
        self.rnd_a_base_other_outside = None

        # rnd_その他の基礎の面積（床下側）, m2
        self.rnd_a_base_other_inside = None

        # rnd_土間床周長（外気側）, m
        self.rnd_l_prm_outside = None

        # rnd_土間床周長（床下側）, m
        self.rnd_l_prm_inside = None

        # rnd_玄関等の土間床等の外周部の長さ（外気側）, m
        self.rnd_l_base_entrance_outside= None

        # rnd_玄関等の土間床等の外周部の長さ（床下側）, m
        self.rnd_l_base_entrance_inside = None

        # rnd_浴室の土間床等の外周部の長さ（外気側）, m
        self.rnd_l_base_bath_outside = None

        # rnd_浴室の土間床等の外周部の長さ（床下側）, m
        self.rnd_l_base_bath_inside = None

        # rnd_玄関等を除く土間床等の外周部の長さ（外気側）, m
        self.rnd_l_base_except_entrance_outside = None

        # rnd_玄関等を除く土間床等の外周部の長さ（床下側）, m
        self.rnd_l_base_except_entrance_inside = None

        # rnd_その他の土間床等の外周部の長さ（外気側）, m
        self.rnd_l_base_other_outside = None

        # rnd_その他の土間床等の外周部の長さ（床下側）, m
        self.rnd_l_base_other_inside = None


# 計算ケース

以下のケースについて検討する。
- 床断熱住宅（外皮に接する浴室床なし）
- 床断熱住宅（浴室床断熱）
- 床断熱住宅（浴室基礎断熱）
- 基礎断熱住宅

In [7]:
hs = [
    House(name='床断熱住宅（外皮に接する浴室床なし）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.NOT_EXIST),
    House(name='床断熱住宅（浴室床断熱）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.FLOOR),
    House(name='床断熱住宅（浴室基礎断熱）', ver=1, floor_ins=FloorIns.FLOOR, bath_ins=BathIns.BASE),
    House(name='基礎断熱住宅', ver=1, floor_ins=FloorIns.BASE, bath_ins=None)
]

# 簡易評価式の算出の前提

部位の面積及び長さ等を計算するにあたっての前提条件を以下に示す。

| 意味 | 記号| 値 | 単位 | 備考 |
| --- | :---: | :---: | :---: | --- |
| 階数 | - | 2 | - | - |
| 床面積の合計 | $A_{F,total}$ | 90.0 | m<sup>2</sup> | - |
| 形状係数(1階) | $SF_1$ | 1.08 | - | - |
| 形状係数(2階) | $SF_2$ | 1.04 | - | - |
| 主開口方位 | - | 南西 | - | - |
| 1階の床面積に対する2階の床面積の比 | $r_{FA}$ | 0.77 | - | 自立循環型住宅プランの比を採用した。 |
| 基礎の高さ(基礎断熱住宅) | $h_{base}$ | 0.5 | m | 基礎の高さとは土間基礎上端から1階床レベルまでの間を言う。<br>(GLからではない。) |
| 基礎の高さ(玄関) | $h_{base,d}$ | 0.18 | m | - |
| 基礎の高さ(浴室) | $h_{base,b}$ | 0.5 | m | - |
| 高さ(1階) | $h_1$ | 2.9 | m | - |
| 高さ(2階) | $h_2$ | 2.7 | m | - |
| ドア(玄関) | $A_{door,1}$ | 1.89 | m<sup>2</sup> | 方位：北西 自立循環型住宅プランの面積を採用 |
| ドア(勝手口) | $A_{door,2}$ | 1.62 | m<sup>2</sup> | 方位：北東 自立循環型住宅プランの面積を採用 |
| 玄関の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,d,MS}$ | 1.365 | m | - |
| 玄関の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,d,OS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(主開口方位側) | $L_{base,b,MS}$ | 1.82 | m | - |
| 浴室の土間床等の周辺部の基礎長さ(その他の方位側) | $L_{base,b,OS}$ | 1.82 | m | - |
| 開口部の面積比率 | $r_{opn}$ | 0.14 | - | 床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、<br>基礎面積を含めない外皮表面積に対する値とする。 |
| 方位別窓面積比率(南西) | $r_{wnd,SW}$ | 0.6862 | - | - |
| 方位別窓面積比率(北西) | $r_{wnd,NW}$ | 0.0721 | - | - |
| 方位別窓面積比率(北東) | $r_{wnd,NE}$ | 0.1097 | - | - |
| 方位別窓面積比率(南東) | $r_{wnd,SE}$ | 0.1320 | - | - |


※基礎の高さについて  
下記を想定すると、根太がけの場合は524～539、根太レスの場合は479～494  
・基礎の高さは400mm（参考：長期優良住宅）  
・土台は105mm又は10mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
・GLから土間基礎上端までは50mm  
あるいは下記を想定すると、根太がけの場合は489～504、根太レスの場合は444～459  
・床下空間高さ（土間基礎上端から大引き下端まで）は330mmを確保（長期優良）  
・大引は90又は105mm  
・根太は45mm（根太がけの場合のみ）  
・床荒板およびフローリングは24mm（12mm＋12mm）  
以上を勘案して、500mmと決めた。

## 階数

階数は2階とする。

##  床面積の合計

$ A_{F,total} = 90.0 $ (m<sup>2</sup>)

In [8]:
for h in hs:
    h.a_f_total = 90.0

## 形状係数

1階: $ SF_1 = 1.08 $  
2階: $ SF_2 = 1.04 $

In [9]:
for h in hs:
    h.sf = [1.08, 1.04]

## 主開口方位

南西とする。

## 1階の床面積に対する2階の床面積の比

自立循環型住宅プランの比を採用する。

$ r_{FA} = 0.77 $

In [10]:
for h in hs:
    h.r_fa = 0.77

## 基礎の高さ

$ h_{base} = 0.5 $ (m) (一般部分)  

$ h_{base,entrance} = 0.18 $ (m) (玄関)

$ h_{base,bath} = 0.5 $ (m) (浴室)

In [11]:
for h in hs:
    h.h_base = 0.5
    h.h_base_entrance = 0.18
    h.h_base_bath = 0.5

## 高さ

$ h_1 = 2.9 $ (1階)  
$ h_2 = 2.7 $ (2階)

In [12]:
for h in hs:
    h.h = [2.9, 2.7]

## ドアの面積

玄関の方位は北西、勝手口の方位は北東とする。自立循環型住宅プランの面積を採用した。

$ A_{door,entrance} = 1.89 $ (m<sup>2</sup>) (玄関)  

$ A_{door,backentrance} = 1.62 $ (m<sup>2</sup>) (勝手口)

In [13]:
for h in hs:
    h.a_door_entrance = 1.89
    h.a_door_backentrance = 1.62

## 玄関の土間床等の周辺部の基礎長さ

$L_{base,entrance,MS}=1.365$(m)(主開口方位側)

$L_{base,entrance,OS}=1.82$(m)(その他方位側)

In [14]:
for h in hs:
    h.l_base_entrance_ms = 1.365
    h.l_base_entrance_os = 1.82

## 浴室の土間床等の周辺部の基礎長さ

$L_{base,bath,MS}=1.82$(m)(主開口方位側)

$L_{base,bath,OS}=1.82$(m)(その他方位側)

In [15]:
for h in hs:
    h.l_base_bath_ms = 1.82
    h.l_base_bath_os = 1.82

## 開口部の面積比率

$r_{opn} = 0.12$

床断熱住宅と基礎断熱住宅とを同じ開口部面積にするために、基礎面積を含めない外皮表面積に対する値とする。

In [16]:
for h in hs:
    h.r_opn = 0.14

## 方位別窓面積比率

南西  
$r_{wnd,SW}=0.6862$  
北西  
$r_{wnd,NW}=0.0721$  
北東  
$r_{wnd,NE}=0.1097$  
南東  
$r_{wnd,SE}=0.1320$

In [17]:
for h in hs:
    h.r_wnd = [0.6862, 0.0721, 0.1097, 0.1320]

# 算出過程

In [18]:
def show_result(prm):
    for h in hs:
        print(str(h.name) + ' : ' + str(prm(h)))

## 床面積(土間床含む)

$ A_{F,1} = \frac{A_{F,total}}{ ( 1 + r_{FA} )} $  

$ A_{F,2} = A_{F,total} - A_{F,1} $

$A_{F,total}$: 床面積の合計(m<sup>2</sup>)  
$r_{FA}$: 1階の床面積に対する2階の床面積の比  
$A_{F,1}$: 1階の床面積(土間床含む)(m<sup>2</sup>)  
$A_{F,2}$: 2階の床面積(m<sup>2</sup>)

In [19]:
def get_a_f(a_f_total: float, r_fa: float) -> List[float]:
    
    a_f_1 = a_f_total / (1.0 + r_fa)
    a_f_2 = a_f_total - a_f_1
    
    return [a_f_1, a_f_2]

In [20]:
for h in hs:
    h.a_f = get_a_f(a_f_total=h.a_f_total, r_fa=h.r_fa)

In [21]:
show_result(lambda h: h.a_f)

床断熱住宅（外皮に接する浴室床なし） : [50.847457627118644, 39.152542372881356]
床断熱住宅（浴室床断熱） : [50.847457627118644, 39.152542372881356]
床断熱住宅（浴室基礎断熱） : [50.847457627118644, 39.152542372881356]
基礎断熱住宅 : [50.847457627118644, 39.152542372881356]


## 周長

$ \begin{align} L_{prm,i} = \sqrt{A_{F,i}} \times 4 \times SF_i \end{align} $

$L_{prm,i}$: $i$階の周長(m)($i=1,2$)  
$SF_i$: $i$階の形状係数($i=1,2$)  

In [22]:
def get_l_prm(a_f: List[float], sf:List[float]) -> List[float]:
    
    return [4 * sf_i * np.sqrt(a_f_i) for a_f_i, sf_i in zip(a_f, sf)]

In [23]:
for h in hs:
    h.l_prm = get_l_prm(a_f=h.a_f, sf=h.sf)

In [24]:
show_result(lambda h: h.l_prm)

床断熱住宅（外皮に接する浴室床なし） : [30.80479821749104, 26.029948852968104]
床断熱住宅（浴室床断熱） : [30.80479821749104, 26.029948852968104]
床断熱住宅（浴室基礎断熱） : [30.80479821749104, 26.029948852968104]
基礎断熱住宅 : [30.80479821749104, 26.029948852968104]


## 長辺の長さ

$ \displaystyle L_{LS,i} = \frac{ \frac{L_{prm,i}}{2} + \sqrt{ \left( \frac{L_{prm,i}}{2} \right)^2 - 4 \times A_{F,i} } }{2} $

$L_{LS,i}$: $i$階の長辺の長さ($i=1,2$)

In [25]:
def get_l_ls(l_prm: List[float], a_f: List[float]) -> List[float]:
    
    return [(l_prm_i / 2 + np.sqrt((l_prm_i / 2)**2 - 4 * a_f_i)) / 2 for l_prm_i, a_f_i in zip(l_prm, a_f)]

In [26]:
for h in hs:
    h.l_ls = get_l_ls(l_prm=h.l_prm, a_f=h.a_f)

In [27]:
show_result(lambda h: h.l_ls)

床断熱住宅（外皮に接する浴室床なし） : [10.609982280729707, 8.29490083566609]
床断熱住宅（浴室床断熱） : [10.609982280729707, 8.29490083566609]
床断熱住宅（浴室基礎断熱） : [10.609982280729707, 8.29490083566609]
基礎断熱住宅 : [10.609982280729707, 8.29490083566609]


## 短辺の長さ

$ \displaystyle L_{SS,i} = \frac{L_{prm,i}}{2} - L_{LS,i} $

$L_{ss,i}$: $i$階の短辺の長さ(m)

In [28]:
def get_l_ss(l_prm: List[float], l_ls: List[float]) -> List[float]:
    
    return [l_prm_i / 2 - l_ls_i for l_prm_i, l_ls_i in zip(l_prm, l_ls)]

In [29]:
for h in hs:
    h.l_ss = get_l_ss(l_prm=h.l_prm, l_ls=h.l_ls)

In [30]:
show_result(lambda h: h.l_ss)

床断熱住宅（外皮に接する浴室床なし） : [4.7924168280158135, 4.720073590817963]
床断熱住宅（浴室床断熱） : [4.7924168280158135, 4.720073590817963]
床断熱住宅（浴室基礎断熱） : [4.7924168280158135, 4.720073590817963]
基礎断熱住宅 : [4.7924168280158135, 4.720073590817963]


## 玄関の土間床等の面積

$ A_{EF,entrance} = L_{base,entrance,MS} \times L_{base,entrance,OS} $

$A_{EF,entrance}$: 玄関の土間床等の面積(m<sup>2</sup>)  
$L_{base,entrance,MS}$: 玄関の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,entrance,OS}$: 玄関の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

In [31]:
def get_a_ef_entrance(l_base_entrance_ms: float, l_base_entrance_os: float) -> float:
    
    return l_base_entrance_ms * l_base_entrance_os

In [32]:
for h in hs:
    h.a_ef_entrance = get_a_ef_entrance(l_base_entrance_ms=h.l_base_entrance_ms, l_base_entrance_os=h.l_base_entrance_os)

In [33]:
show_result(lambda h: h.a_ef_entrance)

床断熱住宅（外皮に接する浴室床なし） : 2.4843
床断熱住宅（浴室床断熱） : 2.4843
床断熱住宅（浴室基礎断熱） : 2.4843
基礎断熱住宅 : 2.4843


## 浴室の土間床等の面積

$ A_{EF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{EF,bath}$: 浴室の土間床等の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

ただし、「床断熱住宅（外皮に接する浴室床なし）」及び「床断熱住宅（浴室床断熱）」の場合は、0.0(m<sup>2</sup>)とする。

In [34]:
def get_a_ef_bath(floor_ins: FloorIns, bath_ins: BathIns, l_base_bath_ms: float, l_base_bath_os: float) -> float:
    
    if floor_ins == FloorIns.FLOOR:
        if bath_ins == BathIns.NOT_EXIST:
            return 0.0
        elif bath_ins == BathIns.FLOOR:
            return 0.0
        elif bath_ins == BathIns.BASE:
            return l_base_bath_ms * l_base_bath_os
        else:
            raise ValueError()
    elif floor_ins == FloorIns.BASE:
        return l_base_bath_ms * l_base_bath_os
    else:
        raise ValueError()

In [35]:
for h in hs:
    h.a_ef_bath = get_a_ef_bath(
        floor_ins=h.floor_ins, bath_ins=h.bath_ins, l_base_bath_ms=h.l_base_bath_ms, l_base_bath_os=h.l_base_bath_os)

In [36]:
show_result(lambda h: h.a_ef_bath)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.3124000000000002
基礎断熱住宅 : 3.3124000000000002


## 玄関・浴室除くその他の土間床等の面積

$ A_{EF,other} = A_{F,1} - A_{EF,entrance} - A_{EF,bath} $

$ A_{EF,other} $: 土間床等の面積(玄関・浴室除く)(m<sup>2</sup>)  

ただし、「床断熱住宅」の場合は、0.0(m<sup>2</sup>)とする。

In [37]:
def get_a_ef_other(floor_ins: FloorIns, a_f: List[float], a_ef_entrance: float, a_ef_bath: float) -> float:
    
    if floor_ins == FloorIns.FLOOR:
        return 0.0
    elif floor_ins == FloorIns.BASE:
        return a_f[0] - a_ef_entrance - a_ef_bath
    else:
        raise ValueError()

In [38]:
for h in hs:
    h.a_ef_other = get_a_ef_other(floor_ins=h.floor_ins, a_f=h.a_f, a_ef_entrance=h.a_ef_entrance, a_ef_bath=h.a_ef_bath)

In [39]:
show_result(lambda h: h.a_ef_other)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 45.05075762711864


## 土間床等の面積の合計

$ A_{EF,total} = A_{EF,entrane} + A_{EF,bath} + A_{EF,other} $

$A_{EF,total}$ : 土間床等の面積の合計(m<sup>2</sup>)

In [40]:
def get_a_ef_total(a_ef_entrance: float, a_ef_bath: float, a_ef_other: float) -> float:
    
    return a_ef_entrance + a_ef_bath + a_ef_other

In [41]:
for h in hs:
    h.a_ef_total = get_a_ef_total(a_ef_entrance=h.a_ef_entrance, a_ef_bath=h.a_ef_bath, a_ef_other=h.a_ef_other)

In [42]:
show_result(lambda h: h.a_ef_total)

床断熱住宅（外皮に接する浴室床なし） : 2.4843
床断熱住宅（浴室床断熱） : 2.4843
床断熱住宅（浴室基礎断熱） : 5.7967
基礎断熱住宅 : 50.847457627118644


## 玄関等の土間床等の外周部の長さ

外気側：  
南西: $L_{base,entrance,SW}=0.0$  
北西: $L_{base,entrance,NW}=L_{base,entrance,OS}$  
北東: $L_{base,entrance,NE}=L_{base,entrance,MS}$  
南東: $L_{base,entrance,SE}=0.0$  

In [43]:
def get_l_base_entrance_outside(l_base_entrance_ms: float, l_base_entrance_os: float) -> Tuple[float]:
    
    return (0.0, l_base_entrance_os, l_base_entrance_ms, 0.0)

In [44]:
for h in hs:
    h.l_base_entrance_outside = get_l_base_entrance_outside(
        l_base_entrance_ms=h.l_base_entrance_ms, l_base_entrance_os=h.l_base_entrance_os)

In [45]:
show_result(lambda h: h.l_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.365, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.365, 0.0)


床下側:  
$
L_{base,entrance,inside} = \left \{
\begin{array}{ll}
    L_{base,entrance,MS} + L_{base,entrance,OS} & \text{ ( floor insulted house ) } \\
    0.0 & \text{ ( insulated house ) }
\end{array}
\right.
$

In [46]:
def get_l_base_entrance_inside(floor_ins: FloorIns, l_base_entrance_ms: float, l_base_entrance_os: float) -> float:
    
    return {
        FloorIns.FLOOR: l_base_entrance_ms + l_base_entrance_os,
        FloorIns.BASE: 0.0
    }[floor_ins]

In [47]:
for h in hs:
    h.l_base_entrance_inside = get_l_base_entrance_inside(
        floor_ins=h.floor_ins, l_base_entrance_ms=h.l_base_entrance_ms, l_base_entrance_os=h.l_base_entrance_os)

In [48]:
show_result(lambda h: h.l_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.185
床断熱住宅（浴室床断熱） : 3.185
床断熱住宅（浴室基礎断熱） : 3.185
基礎断熱住宅 : 0.0


## 浴室の土間床等の外周部の長さ

外気側：  
南西:  $L_{base,bath,SW}=0.0$  
北西:  $L_{base,bath,NW}=L_{base,bath,OS}$  
北東:  $L_{base,bath,NE}=L_{base,bath,MS}$  
南東:  $L_{base,bath,SE}=0.0$

ただし、「床断熱住宅（外皮に接する浴室床なし）」、「床断熱住宅（浴室床断熱）」及び「基礎断熱住宅」の場合はいずれの長さも0mとする。

In [49]:
def get_l_base_bath_outside(floor_ins: FloorIns, bath_ins: BathIns, l_base_bath_ms: float, l_base_bath_os: float) -> Tuple[float]:
    
    if floor_ins == FloorIns.FLOOR:
        if bath_ins == BathIns.NOT_EXIST or bath_ins == BathIns.FLOOR:
            return 0.0, 0.0, 0.0, 0.0
        elif bath_ins == BathIns.BASE:
            return 0.0, l_base_bath_os, l_base_bath_ms, 0.0
        else:
            raise ValueError
    elif floor_ins == FloorIns.BASE:
        return 0.0, l_base_bath_os, l_base_bath_ms, 0.0
    else:
        raise ValueError

In [50]:
for h in hs:
    h.l_base_bath_outside = get_l_base_bath_outside(
        floor_ins=h.floor_ins, bath_ins=h.bath_ins, l_base_bath_ms=h.l_base_bath_ms, l_base_bath_os=h.l_base_bath_os)

In [51]:
show_result(lambda h: h.l_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.82, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.82, 0.0)


室内側:  
$ L_{base,bath,inside} = L_{base,bath,MS} + L_{base,bath,OS} $

ただし、「床断熱住宅（外皮に接する浴室床なし）」、「床断熱住宅（浴室床断熱）」及び「基礎断熱住宅」の場合はいずれの長さも0mとする。

In [52]:
def get_l_base_bath_inside(floor_ins: FloorIns, bath_ins: BathIns, l_base_bath_ms: float, l_base_bath_os: float) -> float:
    
    if floor_ins == FloorIns.FLOOR:
        if bath_ins == BathIns.NOT_EXIST or bath_ins == BathIns.FLOOR:
            return 0.0
        elif bath_ins == BathIns.BASE:
            return l_base_bath_ms + l_base_bath_os
        else:
            raise ValueError
    elif floor_ins == FloorIns.BASE:
        return 0.0
    else:
        raise ValueError

In [53]:
for h in hs:
    h.l_base_bath_inside = get_l_base_bath_inside(
        floor_ins=h.floor_ins, bath_ins=h.bath_ins, l_base_bath_ms=h.l_base_bath_ms, l_base_bath_os=h.l_base_bath_os)

In [54]:
show_result(lambda h: h.l_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.64
基礎断熱住宅 : 0.0


## 玄関・浴室除くその他の土間床等の外周部の長さ

南西: $L_{base,SW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,SW} - L_{base,bath,SW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北西: $L_{base,NW} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,NW} - L_{base,bath,NW} & \text{ ( base insulated house ) }
\end{array}
\right.
$

北東: $L_{base,NE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{LS,1} - L_{base,entrance,NE} - L_{base,bath,NE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

南東: $L_{base,SE} = \left \{
\begin{array}{ll}
    0.0 & \text{ ( floor insulated house ) } \\
    L_{SS,1} - L_{base,entrance,SE} - L_{base,bath,SE} & \text{ ( base insulated house ) }
\end{array}
\right.
$

In [55]:
def get_l_base_other_outside(
    floor_ins: FloorIns, l_ls: Tuple[float], l_ss: Tuple[float], l_base_entrance_outside: Tuple[float], l_base_bath_outside: Tuple[float]
) -> Tuple[float]:
    
    if floor_ins == FloorIns.FLOOR:
        return (0.0, 0.0, 0.0, 0.0)
    elif floor_ins == FloorIns.BASE:
        return (
            l_ls[0] - l_base_entrance_outside[0] - l_base_bath_outside[0],
            l_ss[0] - l_base_entrance_outside[1] - l_base_bath_outside[1],
            l_ls[0] - l_base_entrance_outside[2] - l_base_bath_outside[2],
            l_ss[0] - l_base_entrance_outside[3] - l_base_bath_outside[3]
        )
    else:
        raise ValueError()       

In [56]:
for h in hs:
    h.l_base_other_outside = get_l_base_other_outside(
        floor_ins=h.floor_ins, l_ls=h.l_ls, l_ss=h.l_ss, l_base_entrance_outside=h.l_base_entrance_outside,
        l_base_bath_outside=h.l_base_bath_outside)

In [57]:
show_result(lambda h: h.l_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (10.609982280729707, 1.1524168280158131, 7.424982280729706, 4.7924168280158135)


室内側:  
$ L_{base,inside} = 0.0 $

In [58]:
def get_l_base_other_inside() -> float:
    
    return 0.0

In [59]:
for h in hs:
    h.l_base_other_inside = get_l_base_other_inside()

In [60]:
show_result(lambda h: h.l_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 土間床周長(合計)

南西:   $ L_{prm,SW} = L_{base,SW} + L_{base,entrance,SW} + L_{base,bath,SW} $  
北西:   $ L_{prm,NW} = L_{base,NW} + L_{base,entrance,NW} + L_{base,bath,NW} $  
北東:   $ L_{prm,NE} = L_{base,NE} + L_{base,entrance,NE} + L_{base,bath,NE} $  
南東:   $ L_{prm,SE} = L_{base,SE} + L_{base,entrance,SE} + L_{base,bath,SE} $  

In [61]:
def get_l_prm_outside(
    l_base_outside: Tuple[float], l_base_entrance_outside: Tuple[float], l_base_bath_outside: Tuple[float]) -> Tuple[float]:

    return (
        l_base_outside[0] + l_base_entrance_outside[0] + l_base_bath_outside[0],
        l_base_outside[1] + l_base_entrance_outside[1] + l_base_bath_outside[1],
        l_base_outside[2] + l_base_entrance_outside[2] + l_base_bath_outside[2],
        l_base_outside[3] + l_base_entrance_outside[3] + l_base_bath_outside[3]
    )

In [62]:
for h in hs:
    h.l_prm_outside = get_l_prm_outside(
        l_base_outside=h.l_base_other_outside, l_base_entrance_outside=h.l_base_entrance_outside,
        l_base_bath_outside=h.l_base_bath_outside)

In [63]:
show_result(lambda h: h.l_prm_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.82, 1.365, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 3.64, 3.185, 0.0)
基礎断熱住宅 : (10.609982280729707, 4.7924168280158135, 10.609982280729707, 4.7924168280158135)


室内側: $ L_{prm,inside} = L_{base,inside} + L_{base,entrance,inside} + L_{base,bath,inside} $

In [64]:
def get_l_prm_inside(l_base_inside: float, l_base_d_inside: float, l_base_b_inside: float) -> float:

    return l_base_inside + l_base_d_inside + l_base_b_inside

In [65]:
for h in hs:
    h.l_prm_inside = get_l_prm_inside(
        l_base_inside=h.l_base_other_inside, l_base_d_inside=h.l_base_entrance_inside,
        l_base_b_inside=h.l_base_bath_inside)

In [66]:
show_result(lambda h: h.l_prm_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.185
床断熱住宅（浴室床断熱） : 3.185
床断熱住宅（浴室基礎断熱） : 6.825
基礎断熱住宅 : 0.0


## 浴室の床断熱した床の面積

$ A_{IF,bath} = L_{base,bath,MS} \times L_{base,bath,OS} $

$A_{IF,bath}$: 浴室の床断熱した床の面積(m<sup>2</sup>)  
$L_{base,bath,MS}$: 浴室の土間床等の周辺部の基礎長さ(主開口方位側)(m)  
$L_{base,bath,OS}$: 浴室の土間床等の周辺部の基礎長さ(その他の方位側)(m)  

ただし、「床断熱住宅（外皮に接する浴室床なし）」及び「床断熱住宅（浴室床断熱）」の場合のみ。  
その他の場合は、0.0(m<sup>2</sup>)とする。

In [67]:
def get_a_if_bath(floor_ins: FloorIns, bath_ins: BathIns, l_base_bath_ms: float, l_base_bath_os: float) -> float:
    
    if floor_ins == FloorIns.FLOOR:
        if bath_ins == BathIns.NOT_EXIST:
            return l_base_bath_ms * l_base_bath_os
        elif bath_ins == BathIns.FLOOR:
            return l_base_bath_ms * l_base_bath_os
        elif bath_ins == BathIns.BASE:
            return 0.0
        else:
            raise ValueError()
    elif floor_ins == FloorIns.BASE:
        return 0.0
    else:
        raise ValueError()

In [68]:
for h in hs:
    h.a_if_bath = get_a_if_bath(
        floor_ins=h.floor_ins, bath_ins=h.bath_ins, l_base_bath_ms=h.l_base_bath_ms, l_base_bath_os=h.l_base_bath_os)

In [69]:
show_result(lambda h: h.a_if_bath)

床断熱住宅（外皮に接する浴室床なし） : 3.3124000000000002
床断熱住宅（浴室床断熱） : 3.3124000000000002
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 浴室除くその他の床断熱した床の面積

$ A_{IF} = A_{F,1} - A_{EF,total} - A_{IF,bath} $ 

$A_{IF}$: 床断熱した床の面積(m<sup>2</sup>)

In [70]:
def get_a_if(a_f: List[float], a_ef_total: float, a_if_bath: float) -> float:
    return a_f[0] - a_ef_total - a_if_bath

In [71]:
for h in hs:
    h.a_if = get_a_if(a_f=h.a_f, a_ef_total=h.a_ef_total, a_if_bath=h.a_if_bath)

In [72]:
show_result(lambda h: h.a_if)

床断熱住宅（外皮に接する浴室床なし） : 45.05075762711864
床断熱住宅（浴室床断熱） : 45.05075762711864
床断熱住宅（浴室基礎断熱） : 45.05075762711864
基礎断熱住宅 : 0.0


## 天井・屋根(下屋を含む)

天井・屋根(下屋を含む)の面積は、オーバーハング床を想定しない限り、1階の床面積(土間床含む)と等しい。

$ A_{roof} = A_{F,1}  $ 

$A_{roof}$: 天井・屋根(下屋を含む)の面積 (m<sup>2</sup>)

In [73]:
def get_a_roof(a_f: List[float]) -> float:
    
    return a_f[0]

In [74]:
for h in hs:
    h.a_roof = get_a_roof(h.a_f)

In [75]:
show_result(lambda h: h.a_roof)

床断熱住宅（外皮に接する浴室床なし） : 50.847457627118644
床断熱住宅（浴室床断熱） : 50.847457627118644
床断熱住宅（浴室基礎断熱） : 50.847457627118644
基礎断熱住宅 : 50.847457627118644


## 基礎(玄関)の面積

南西:  $ A_{base,entrance,SW} = L_{base,entrance,SW} \times h_{base,entrance} $  
北西:  $ A_{base,entrance,NW} = L_{base,entrance,NW} \times h_{base,entrance} $  
北東:  $ A_{base,entrance,NE} = L_{base,entrance,NE} \times h_{base,entrance} $  
南東:  $ L_{base,entrance,SE} = L_{base,entrance,SE} \times h_{base,entrance} $  

$A_{base,entrance,SW}$: 基礎(玄関)(南西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NW}$: 基礎(玄関)(北西)の面積(m<sup>2</sup>)  
$A_{base,entrance,NE}$: 基礎(玄関)(北東)の面積(m<sup>2</sup>)  
$A_{base,entrance,SE}$: 基礎(玄関)(南東)の面積(m<sup>2</sup>)  

In [76]:
def get_a_base_entrance_outside(l_base_entrance_outside: Tuple[float], h_base_entrance: float):
    
    return (
        l_base_entrance_outside[0] * h_base_entrance,
        l_base_entrance_outside[1] * h_base_entrance,
        l_base_entrance_outside[2] * h_base_entrance,
        l_base_entrance_outside[3] * h_base_entrance
    )

In [77]:
for h in hs:
    
    h.a_base_entrance_outside = get_a_base_entrance_outside(
        l_base_entrance_outside=h.l_base_entrance_outside, h_base_entrance=h.h_base_entrance)

In [78]:
show_result(lambda h: h.a_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.3276, 0.2457, 0.0)
基礎断熱住宅 : (0.0, 0.3276, 0.2457, 0.0)


室内側:  $ L_{base,entrance,inside} = L_{base,entrance,inside} \times h_{base,entrance} $

$A_{base,entrance,inside}$: 基礎(玄関)(室内側)の面積(m<sup>2</sup>)  

In [79]:
def get_a_base_entrance_inside(l_base_entrance_inside: float, h_base_entrance: float):
    
    return l_base_entrance_inside * h_base_entrance

In [80]:
for h in hs:
    h.a_base_entrance_inside = get_a_base_entrance_inside(
        l_base_entrance_inside=h.l_base_entrance_inside, h_base_entrance=h.h_base_entrance)

In [81]:
show_result(lambda h: h.a_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.5733
床断熱住宅（浴室床断熱） : 0.5733
床断熱住宅（浴室基礎断熱） : 0.5733
基礎断熱住宅 : 0.0


## 基礎(浴室)の面積

南西:  $ A_{base,bath,SW} = L_{base,bath,SW} \times h_{base,bath} $  
北西:  $ A_{base,bath,NW} = L_{base,bath,NW} \times h_{base,bath} $  
北東:  $ A_{base,bath,NE} = L_{base,bath,NE} \times h_{base,bath} $  
南東:  $ L_{base,bath,SE} = L_{base,bath,SE} \times h_{base,bath} $  

$A_{base,bath,SW}$: 基礎(浴室)(南西)の面積(m<sup>2</sup>)  
$A_{base,bath,NW}$: 基礎(浴室)(北西)の面積(m<sup>2</sup>)  
$A_{base,bath,NE}$: 基礎(浴室)(北東)の面積(m<sup>2</sup>)  
$A_{base,bath,SE}$: 基礎(浴室)(南東)の面積(m<sup>2</sup>)  

In [82]:
def get_a_base_bath_outside(l_base_bath: Tuple[float], h_base_bath: float) -> Tuple[float]:
    
    return (
        l_base_bath[0] * h_base_bath,
        l_base_bath[1] * h_base_bath,
        l_base_bath[2] * h_base_bath,
        l_base_bath[3] * h_base_bath
    )

In [83]:
for h in hs:
    h.a_base_bath_outside = get_a_base_bath_outside(l_base_bath=h.l_base_bath_outside, h_base_bath=h.h_base_bath)

In [84]:
show_result(lambda h: h.a_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.91, 0.91, 0.0)
基礎断熱住宅 : (0.0, 0.91, 0.91, 0.0)


室内側:  $ L_{base,bath,inside} = L_{base,bath,inside} \times h_{base,bath} $

$A_{base,bath,inside}$: 基礎(浴室)(室内側)の面積(m<sup>2</sup>)  

In [85]:
def get_a_base_bath_inside(l_base_bath_inside: float, h_base_bath: float) -> float:
    
    return l_base_bath_inside * h_base_bath

In [86]:
for h in hs:
    h.a_base_bath_inside = get_a_base_bath_inside(l_base_bath_inside=h.l_base_bath_inside, h_base_bath=h.h_base_bath)

In [87]:
show_result(lambda h: h.a_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 1.82
基礎断熱住宅 : 0.0


## 基礎断熱部分(玄関・浴室除く)の面積

南西: $ A_{base,SW} = L_{base,SW} \times h_{base} $  
北西:  $ A_{base,NW} = L_{base,NW} \times h_{base} $  
北東:  $ A_{base,NE} = L_{base,NE} \times h_{base} $  
南東:  $ A_{base,SE} = L_{base,SE} \times h_{base} $  

$A_{base,SW}$: 基礎断熱部分(玄関・浴室除く)(南西)の面積(m<sup>2</sup>)  
$A_{base,NW}$: 基礎断熱部分(玄関・浴室除く)(北西)の面積(m<sup>2</sup>)  
$A_{base,NE}$: 基礎断熱部分(玄関・浴室除く)(北東)の面積(m<sup>2</sup>)  
$A_{base,SE}$: 基礎断熱部分(玄関・浴室除く)(南東)の面積(m<sup>2</sup>)  

In [88]:
def get_a_base_other_outside(l_base_outside: Tuple[float], h_base: float) -> Tuple[float]:
    
    return (
        l_base_outside[0] * h_base,
        l_base_outside[1] * h_base,
        l_base_outside[2] * h_base,
        l_base_outside[3] * h_base
    )

In [89]:
for h in hs:
    h.a_base_other_outside = get_a_base_other_outside(l_base_outside=h.l_base_other_outside, h_base=h.h_base)

In [90]:
show_result(lambda h: h.a_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (5.304991140364853, 0.5762084140079066, 3.712491140364853, 2.3962084140079067)


室内側:  $ A_{base,inside} = 0.0 $

$A_{base,inside}$: 基礎断熱部分(玄関・浴室除く)(室内側)の面積(m<sup>2</sup>)  

In [91]:
def get_a_base_other_inside() -> float:
    
    return 0.0

In [92]:
for h in hs:
    h.a_base_other_inside = get_a_base_other_inside()

In [93]:
show_result(lambda h: h.a_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 基礎断熱部分の面積の合計

南西:  $ A_{prm,SW} = A_{base,SW} + A_{base,entrance,SW} + A_{base,b,SW} $  
北西:  $ A_{prm,NW} = A_{base,NW} + A_{base,entrance,NW} + A_{base,b,NW} $  
北東:  $ A_{prm,NE} = A_{base,NE} + A_{base,entrance,NE} + A_{base,b,NE} $  
南東:  $ A_{prm,SE} = A_{base,SE} + A_{base,entrance,SE} + A_{base,b,SE} $  

$A_{prm,SW}$: 基礎断熱部分(南西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NW}$: 基礎断熱部分(北西)の面積の合計(m<sup>2</sup>)  
$A_{prm,NE}$: 基礎断熱部分(北東)の面積の合計(m<sup>2</sup>)  
$A_{prm,SE}$: 基礎断熱部分(南東)の面積の合計(m<sup>2</sup>)  

In [94]:
def get_a_base_total_outside(a_base: Tuple[float], a_base_entrance: Tuple[float], a_base_bath: Tuple[float]) -> Tuple[float]:
    
    return (
        a_base[0] + a_base_entrance[0] + a_base_bath[0],
        a_base[1] + a_base_entrance[1] + a_base_bath[1],
        a_base[2] + a_base_entrance[2] + a_base_bath[2],
        a_base[3] + a_base_entrance[3] + a_base_bath[3]
    )

In [95]:
for h in hs:
    h.a_base_total_outside = get_a_base_total_outside(
        a_base=h.a_base_other_outside, a_base_entrance=h.a_base_entrance_outside, a_base_bath=h.a_base_bath_outside)

In [96]:
show_result(lambda h: h.a_base_total_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.3276, 0.2457, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.2376, 1.1557, 0.0)
基礎断熱住宅 : (5.304991140364853, 1.8138084140079065, 4.868191140364853, 2.3962084140079067)


室内側:  $ A_{prm,IS} = A_{base,IS} + A_{base,entrance,IS} + A_{base,b,IS} $

$A_{prm,IS}$: 基礎断熱部分(室内側)の面積の合計(m<sup>2</sup>)  

In [97]:
def get_a_base_total_inside(a_base_inside: float, a_base_entrance_inside: float, a_base_bath_inside: float) -> float:
    
    return a_base_inside + a_base_entrance_inside + a_base_bath_inside

In [98]:
for h in hs:
    h.a_base_total_inside = get_a_base_total_inside(
        a_base_inside=h.a_base_other_inside, a_base_entrance_inside=h.a_base_entrance_inside, a_base_bath_inside=h.a_base_bath_inside)

In [99]:
show_result(lambda h: h.a_base_total_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.5733
床断熱住宅（浴室床断熱） : 0.5733
床断熱住宅（浴室基礎断熱） : 2.3933
基礎断熱住宅 : 0.0


## 壁面積(開口部含む)

1階:  
南西: $ A_{srf,1,SW} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,NW} = L_{SS,1} \times h_1 $  
北東: $ A_{srf,1,NE} = L_{LS,1} \times h_1 $  
北西: $ A_{srf,1,SE} = L_{SS,1} \times h_1 $

2階:  
南西: $ A_{srf,2,SW} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,NW} = L_{SS,2} \times h_2 $  
北東: $ A_{srf,2,NE} = L_{LS,2} \times h_2 $  
北西: $ A_{srf,2,SE} = L_{SS,2} \times h_2 $

In [100]:
def get_a_srf(l_ls: List[float], l_ss: List[float], h: List[float]) -> [Tuple[float], Tuple[float]]:

    return [
        (l_ls[0] * h[0], l_ss[0] * h[0], l_ls[0] * h[0], l_ss[0] * h[0]),
        (l_ls[1] * h[1], l_ss[1] * h[1], l_ls[1] * h[1], l_ss[1] * h[1])
    ]

In [101]:
for h in hs:
    h.a_srf = get_a_srf(l_ls=h.l_ls, l_ss=h.l_ss, h=h.h)

In [102]:
show_result(lambda h: h.a_srf)

床断熱住宅（外皮に接する浴室床なし） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
床断熱住宅（浴室床断熱） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
床断熱住宅（浴室基礎断熱） : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]
基礎断熱住宅 : [(30.768948614116148, 13.898008801245858, 30.768948614116148, 13.898008801245858), (22.396232256298443, 12.7441986952085, 22.396232256298443, 12.7441986952085)]


## 外皮面積の合計(基礎の面積を除く)

$ A'_{env} = A_{IF} + A_{IF,bath} + A_{EF,total} + A_{roof} \\
+ A_{srf,1,SW} + A_{srf,1,NW} + A_{srf,1,NE} + A_{srf,1,SE} \\
+ A_{srf,2,SW} + A_{srf,2,NW} + A_{srf,2,NE} + A_{srf,2,SE} $

$ A'_{env}$: 外皮面積の合計(基礎の面積を除く)(m<sup>2</sup>)

In [103]:
def get_a_dash_env(a_if: float, a_if_bath: float, a_ef_total: float, a_roof: float, a_srf: [Tuple[float], Tuple[float]]) -> float:
    
    return a_if + a_if_bath + a_ef_total + a_roof + sum(a_srf[0]) + sum(a_srf[1])

In [104]:
for h in hs:
    h.a_dash_env = get_a_dash_env(a_if=h.a_if, a_if_bath=h.a_if_bath, a_ef_total=h.a_ef_total, a_roof=h.a_roof, a_srf=h.a_srf)

In [105]:
show_result(lambda h: h.a_dash_env)

床断熱住宅（外皮に接する浴室床なし） : 261.30969198797516
床断熱住宅（浴室床断熱） : 261.30969198797516
床断熱住宅（浴室基礎断熱） : 261.30969198797516
基礎断熱住宅 : 261.30969198797516


## 外皮面積の合計(m<sup>2</sup>)

$ A_{env} = A'_{env} + A_{prm,SW} + A_{prm,NW} + A_{prm,NE} + A_{prm,SE} + A_{prm,IS} $

$ A_{env} $: 外皮面積の合計(m<sup>2</sup>)

In [106]:
def get_a_env(a_dash_env: float, a_prm_outside: Tuple[float], a_prm_inside: float) -> float:
    
    return a_dash_env + sum(a_prm_outside) + a_prm_inside

In [107]:
for h in hs:
    h.a_env = get_a_env(a_dash_env=h.a_dash_env, a_prm_outside=h.a_base_total_outside, a_prm_inside=h.a_base_total_inside)

In [108]:
show_result(lambda h: h.a_env)

床断熱住宅（外皮に接する浴室床なし） : 262.4562919879752
床断熱住宅（浴室床断熱） : 262.4562919879752
床断熱住宅（浴室基礎断熱） : 266.0962919879752
基礎断熱住宅 : 275.6928910967207


## 開口部の面積の合計

$ A_{opn} = A'_{env} \times r_{opn} $

$A_{opn}$: 開口部の面積の合計(m<sup>2</sup>)

In [109]:
def get_a_opn(a_dash_env: float, r_opn: float) -> float:
    
    return a_dash_env * r_opn

In [110]:
for h in hs:
    h.a_opn = get_a_opn(a_dash_env=h.a_dash_env, r_opn=h.r_opn)

In [111]:
show_result(lambda h: h.a_opn)

床断熱住宅（外皮に接する浴室床なし） : 36.58335687831652
床断熱住宅（浴室床断熱） : 36.58335687831652
床断熱住宅（浴室基礎断熱） : 36.58335687831652
基礎断熱住宅 : 36.58335687831652


## 窓の面積の合計

$ A_{wnd,total} = A_{opn} - A_{door,entrance} - A_{door,backentrance} $

$A_{wnd,total}$: 窓の面積(m<sup>2</sup>)

In [112]:
def get_a_wnd_total(a_opn: float, a_door_entrance: float, a_door_backentrance: float) -> float:
    
    return a_opn - a_door_entrance - a_door_backentrance

In [113]:
for h in hs:
    h.a_wnd_total = get_a_wnd_total(a_opn=h.a_opn, a_door_entrance=h.a_door_entrance, a_door_backentrance=h.a_door_backentrance)

In [114]:
show_result(lambda h: h.a_wnd_total)

床断熱住宅（外皮に接する浴室床なし） : 33.073356878316524
床断熱住宅（浴室床断熱） : 33.073356878316524
床断熱住宅（浴室基礎断熱） : 33.073356878316524
基礎断熱住宅 : 33.073356878316524


## 窓の面積

南西側: $ A_{wnd,SW} = A_{wnd} \times r_{wnd,SW} $  
北西側: $ A_{wnd,NW} = A_{wnd} \times r_{wnd,NW} $  
北東側: $ A_{wnd,NE} = A_{wnd} \times r_{wnd,NE} $  
南東側: $ A_{wnd,SE} = A_{wnd} \times r_{wnd,SE} $  

$ A_{wnd,SW}$: 窓の面積(南西)(m<sup>2</sup>)  
$ A_{wnd,NW}$: 窓の面積(北西)(m<sup>2</sup>)  
$ A_{wnd,NE}$: 窓の面積(北東)(m<sup>2</sup>)  
$ A_{wnd,SE}$: 窓の面積(南東)(m<sup>2</sup>)  

In [115]:
def get_a_wnd(a_wnd_total: float, r_wnd: Tuple[float]) -> Tuple[float]:
    
    return a_wnd_total * r_wnd[0], a_wnd_total * r_wnd[1], a_wnd_total * r_wnd[2], a_wnd_total * r_wnd[3]

In [116]:
for h in hs:
    h.a_wnd = get_a_wnd(a_wnd_total=h.a_wnd_total, r_wnd=h.r_wnd)

In [117]:
show_result(lambda h: h.a_wnd)

床断熱住宅（外皮に接する浴室床なし） : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)
床断熱住宅（浴室床断熱） : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)
床断熱住宅（浴室基礎断熱） : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)
基礎断熱住宅 : (22.6949374899008, 2.3845890309266213, 3.628147249551323, 4.365683107937781)


## ドアの面積

南西側: $ A_{door,SW} = 0.0 $  
北西側: $ A_{door,NW} = A_{door,entrance} $   
北東側: $ A_{door,NE} = A_{door,backentrance} $  
南東側: $ A_{door,SE} = 0.0 $  

$A_{door}$: ドアの面積(m<sup>2</sup>)

In [118]:
def get_a_door(a_door_entrance: float, a_door_backentrance: float) -> Tuple[float]:
    
    return 0.0, a_door_entrance, a_door_backentrance, 0.0

In [119]:
for h in hs:
    h.a_door = get_a_door(a_door_entrance=h.a_door_entrance, a_door_backentrance=h.a_door_backentrance)

In [120]:
show_result(lambda h: h.a_door_entrance)

床断熱住宅（外皮に接する浴室床なし） : 1.89
床断熱住宅（浴室床断熱） : 1.89
床断熱住宅（浴室基礎断熱） : 1.89
基礎断熱住宅 : 1.89


## 壁の面積

南西側: $ A_{wall,SW} = A_{srf,1,SW} + A_{srf,2,SW} - A_{wnd,SW} - A_{door,SW} $  
北西側: $ A_{wall,NW} = A_{srf,1,NW} + A_{srf,2,NW} - A_{wnd,NW} - A_{door,NW} $  
北東側: $ A_{wall,NE} = A_{srf,1,NE} + A_{srf,2,NE} - A_{wnd,NE} - A_{door,NE} $  
南東側: $ A_{wall,SE} = A_{srf,1,SE} + A_{srf,2,SE} - A_{wnd,SE} - A_{door,SE} $  

$ A_{wall}$: 壁の面積(m<sup>2</sup>)

In [121]:
def get_a_wall(a_srf: [Tuple[float], Tuple[float]], a_wnd: Tuple[float], a_door: Tuple[float]) -> Tuple[float]:
    
    return (
        a_srf[0][0] + a_srf[1][0] - a_wnd[0] - a_door[0],
        a_srf[0][1] + a_srf[1][1] - a_wnd[1] - a_door[1],
        a_srf[0][2] + a_srf[1][2] - a_wnd[2] - a_door[2],
        a_srf[0][3] + a_srf[1][3] - a_wnd[3] - a_door[3]
    )        

In [122]:
for h in hs:
    h.a_wall = get_a_wall(a_srf=h.a_srf, a_wnd=h.a_wnd, a_door=h.a_door)

In [123]:
show_result(lambda h: h.a_wall)

床断熱住宅（外皮に接する浴室床なし） : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)
床断熱住宅（浴室床断熱） : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)
床断熱住宅（浴室基礎断熱） : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)
基礎断熱住宅 : (30.47024338051379, 22.367618465527734, 47.917033620863265, 22.276524388516577)


# 結果

## 床面積の合計(m<sup>2</sup>)

In [124]:
for h in hs:
    h.rnd_a_f_total = round2(h.a_f_total)

In [125]:
show_result(lambda h: h.rnd_a_f_total)

床断熱住宅（外皮に接する浴室床なし） : 90.0
床断熱住宅（浴室床断熱） : 90.0
床断熱住宅（浴室基礎断熱） : 90.0
基礎断熱住宅 : 90.0


## 外皮の部位の面積の合計(m<sup>2</sup>)

In [126]:
for h in hs:
    h.rnd_a_env = round2(h.a_env)

In [127]:
show_result(lambda h: h.rnd_a_env)

床断熱住宅（外皮に接する浴室床なし） : 262.46
床断熱住宅（浴室床断熱） : 262.46
床断熱住宅（浴室基礎断熱） : 266.1
基礎断熱住宅 : 275.69


## 屋根又は天井(下屋を含む)の面積(m<sup>2</sup>)

In [128]:
for h in hs:
    h.rnd_a_roof = round2(h.a_roof)

In [129]:
show_result(lambda h: h.rnd_a_roof)

床断熱住宅（外皮に接する浴室床なし） : 50.85
床断熱住宅（浴室床断熱） : 50.85
床断熱住宅（浴室基礎断熱） : 50.85
基礎断熱住宅 : 50.85


## 壁の面積(m<sup>2</sup>) 

In [130]:
for h in hs:
    h.rnd_a_wall = round2_direction(h.a_wall)

In [131]:
show_result(lambda h: h.rnd_a_wall)

床断熱住宅（外皮に接する浴室床なし） : (30.47, 22.37, 47.92, 22.28)
床断熱住宅（浴室床断熱） : (30.47, 22.37, 47.92, 22.28)
床断熱住宅（浴室基礎断熱） : (30.47, 22.37, 47.92, 22.28)
基礎断熱住宅 : (30.47, 22.37, 47.92, 22.28)


## 窓の面積(m<sup>2</sup>) 

In [132]:
for h in hs:
    h.rnd_a_wnd = round2_direction(h.a_wnd)

In [133]:
show_result(lambda h: h.rnd_a_wnd)

床断熱住宅（外皮に接する浴室床なし） : (22.69, 2.38, 3.63, 4.37)
床断熱住宅（浴室床断熱） : (22.69, 2.38, 3.63, 4.37)
床断熱住宅（浴室基礎断熱） : (22.69, 2.38, 3.63, 4.37)
基礎断熱住宅 : (22.69, 2.38, 3.63, 4.37)


## ドアの面積(m<sup>2</sup>)

In [134]:
for h in hs:
    h.rnd_a_door = round2_direction(h.a_door)

In [135]:
show_result(lambda h: h.rnd_a_door)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.89, 1.62, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.89, 1.62, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.89, 1.62, 0.0)
基礎断熱住宅 : (0.0, 1.89, 1.62, 0.0)


## 浴室の床断熱した床の面積

In [136]:
for h in hs:
    h.rnd_a_if_bath = round2(h.a_if_bath)

In [137]:
show_result(lambda h: h.rnd_a_if_bath)

床断熱住宅（外皮に接する浴室床なし） : 3.31
床断熱住宅（浴室床断熱） : 3.31
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## （浴室除くその他の）床断熱した床の面積(m<sup>2</sup>)  

In [138]:
for h in hs:
    h.rnd_a_if = round2(h.a_if)

In [139]:
show_result(lambda h: h.rnd_a_if)

床断熱住宅（外皮に接する浴室床なし） : 45.05
床断熱住宅（浴室床断熱） : 45.05
床断熱住宅（浴室基礎断熱） : 45.05
基礎断熱住宅 : 0.0


## 土間床等の面積の合計(m<sup>2</sup>)  

参考値。

In [140]:
for h in hs:
    h.rnd_a_ef_total = round2(h.a_ef_total)

In [141]:
show_result(lambda h: h.rnd_a_ef_total)

床断熱住宅（外皮に接する浴室床なし） : 2.48
床断熱住宅（浴室床断熱） : 2.48
床断熱住宅（浴室基礎断熱） : 5.8
基礎断熱住宅 : 50.85


## 浴室の土間床等の面積(m<sup>2</sup>)

In [142]:
for h in hs:
    h.rnd_a_ef_bath = round2(h.a_ef_bath)

In [143]:
show_result(lambda h: h.rnd_a_ef_bath)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.31
基礎断熱住宅 : 3.31


## 基礎の面積の合計(m<sup>2</sup>)  

従来はこの値を表示していたが、2017.07から、玄関とそれ以外に分離させる。  
現在ではこの値は表示していない。

In [144]:
for h in hs:
    h.rnd_a_base_total_outside = round2_direction(h.a_base_total_outside)
    h.rnd_a_base_total_inside = round2(h.a_base_total_inside)

In [145]:
show_result(lambda h: h.rnd_a_base_total_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.33, 0.25, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.33, 0.25, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.24, 1.16, 0.0)
基礎断熱住宅 : (5.3, 1.81, 4.87, 2.4)


In [146]:
show_result(lambda h: h.rnd_a_base_total_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.57
床断熱住宅（浴室床断熱） : 0.57
床断熱住宅（浴室基礎断熱） : 2.39
基礎断熱住宅 : 0.0


## 玄関等の基礎の面積(m<sup>2</sup>)  

In [147]:
for h in hs:
    h.rnd_a_base_entrance_outside = round2_direction(h.a_base_entrance_outside)
    h.rnd_a_base_entrance_inside = round2(h.a_base_entrance_inside)

In [148]:
show_result(lambda h: h.rnd_a_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.33, 0.25, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.33, 0.25, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.33, 0.25, 0.0)
基礎断熱住宅 : (0.0, 0.33, 0.25, 0.0)


In [149]:
show_result(lambda h: h.rnd_a_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.57
床断熱住宅（浴室床断熱） : 0.57
床断熱住宅（浴室基礎断熱） : 0.57
基礎断熱住宅 : 0.0


## 浴室の基礎の面積(m<sup>2</sup>)  

In [150]:
for h in hs:
    h.rnd_a_base_bath_outside = round2_direction(h.a_base_bath_outside)
    h.rnd_a_base_bath_inside = round2(h.a_base_bath_inside)

In [151]:
show_result(lambda h: h.rnd_a_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.91, 0.91, 0.0)
基礎断熱住宅 : (0.0, 0.91, 0.91, 0.0)


In [152]:
show_result(lambda h: h.rnd_a_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 1.82
基礎断熱住宅 : 0.0


## 玄関等を除く基礎の面積(m<sup>2</sup>)  

2017.07まで「基礎の面積の合計」のみを表示していたが、2017.07より、「基礎の面積(玄関)」と「基礎の面積(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「基礎の面積(玄関除く)」は、「基礎の面積(その他)」と「基礎の面積(浴室)」の合計とすべきところである。しかし、「基礎の面積(玄関)」と「基礎の面積(浴室)」、「基礎の面積(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「基礎の面積の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「基礎の面積の合計」を小数点第二位まで四捨五入した値から、「基礎の面積(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [153]:
for h in hs:
    h.rnd_a_base_except_entrance_outside = round2_direction((
        round2(h.a_base_total_outside[0]) - round2(h.a_base_entrance_outside[0]),
        round2(h.a_base_total_outside[1]) - round2(h.a_base_entrance_outside[1]),
        round2(h.a_base_total_outside[2]) - round2(h.a_base_entrance_outside[2]),
        round2(h.a_base_total_outside[3]) - round2(h.a_base_entrance_outside[3])
    ))
    h.rnd_a_base_except_entrance_inside = round2(round2(h.a_base_total_inside) - round2(h.a_base_entrance_inside))

In [154]:
show_result(lambda h: h.rnd_a_base_except_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.91, 0.91, 0.0)
基礎断熱住宅 : (5.3, 1.48, 4.62, 2.4)


In [155]:
show_result(lambda h: h.rnd_a_base_except_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 1.82
基礎断熱住宅 : 0.0


## その他の基礎の面積(m<sup>2</sup>)

In [156]:
for h in hs:
    h.rnd_a_base_other_outside = round2_direction((
        round2(h.a_base_total_outside[0]) - round2(h.a_base_entrance_outside[0]) - round2(h.a_base_bath_outside[0]),
        round2(h.a_base_total_outside[1]) - round2(h.a_base_entrance_outside[1]) - round2(h.a_base_bath_outside[1]),
        round2(h.a_base_total_outside[2]) - round2(h.a_base_entrance_outside[2]) - round2(h.a_base_bath_outside[2]),
        round2(h.a_base_total_outside[3]) - round2(h.a_base_entrance_outside[3]) - round2(h.a_base_bath_outside[3])
    ))
    h.rnd_a_base_other_inside = round2(round2(h.a_base_total_inside) - round2(h.a_base_entrance_inside) - round2(h.a_base_bath_inside))

In [157]:
show_result(lambda h: h.rnd_a_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (5.3, 0.57, 3.71, 2.4)


In [158]:
show_result(lambda h: h.rnd_a_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


## 土間床周長の合計(m)

従来はこの値を表示していたが、2017.07から、玄関とそれ以外に分離させる。  
現在ではこの値は表示していない。

In [159]:
for h in hs:
    h.rnd_l_prm_outside = round2_direction(h.l_prm_outside)
    h.rnd_l_prm_inside = round2(h.l_prm_inside)

In [160]:
show_result(lambda h: h.rnd_l_prm_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.82, 1.37, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.82, 1.37, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 3.64, 3.19, 0.0)
基礎断熱住宅 : (10.61, 4.79, 10.61, 4.79)


In [161]:
show_result(lambda h: h.rnd_l_prm_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.19
床断熱住宅（浴室床断熱） : 3.19
床断熱住宅（浴室基礎断熱） : 6.83
基礎断熱住宅 : 0.0


## 玄関等の土間床等の外周部の長さ(m)

In [162]:
for h in hs:
    h.rnd_l_base_entrance_outside = round2_direction(h.l_base_entrance_outside)
    h.rnd_l_base_entrance_inside = round2(h.l_base_entrance_inside)

In [163]:
show_result(lambda h: h.rnd_l_base_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 1.82, 1.37, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 1.82, 1.37, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.37, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.37, 0.0)


In [164]:
show_result(lambda h: h.rnd_l_base_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 3.19
床断熱住宅（浴室床断熱） : 3.19
床断熱住宅（浴室基礎断熱） : 3.19
基礎断熱住宅 : 0.0


## 浴室の土間床等の外周部の長さ

In [165]:
for h in hs:
    h.rnd_l_base_bath_outside = round2_direction(h.l_base_bath_outside)
    h.rnd_l_base_bath_inside = round2(h.l_base_bath_inside)

In [166]:
show_result(lambda h: h.rnd_l_base_bath_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.82, 0.0)
基礎断熱住宅 : (0.0, 1.82, 1.82, 0.0)


In [167]:
show_result(lambda h: h.rnd_l_base_bath_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.64
基礎断熱住宅 : 0.0


## 玄関等を除く土間床等の外周部の長さ(m)

2017.07まで「土間床周長の合計」のみを表示していたが、2017.07より、「土間床周長(玄関)」と「土間床周長(玄関除く)」に分けて記載することになった。  
簡易法の表記上、基礎の面積は、「玄関」と「（玄関以外の）その他」としてまとめる。  
本来であれば、「土間床周長(玄関除く)」は、「土間床周長(その他)」と「土間床周長(浴室)」の合計とすべきところである。しかし、「土間床周長(玄関)」と「土間床周長(浴室)」、「土間床周長(その他)」をそれぞれ小数点第二位まで四捨五入して合計した値は、「土間床周長の合計」を小数点第二位まで四捨五入した値に一致しない。そこで、「土間床周長の合計」を小数点第二位まで四捨五入した値から、「土間床周長(玄関)」を小数点第二位まで四捨五入した値を減じることにより、従来評価に変更がでないように対処する。

In [168]:
for h in hs:
    h.rnd_l_base_except_entrance_outside = round2_direction((
        round2(h.l_prm_outside[0]) - round2(h.l_base_entrance_outside[0]),
        round2(h.l_prm_outside[1]) - round2(h.l_base_entrance_outside[1]),
        round2(h.l_prm_outside[2]) - round2(h.l_base_entrance_outside[2]),
        round2(h.l_prm_outside[3]) - round2(h.l_base_entrance_outside[3])
    ))
    h.rnd_l_base_except_entrance_inside = round2(
        round2(h.l_prm_inside) - round2(h.l_base_entrance_inside)
    )

In [169]:
show_result(lambda h: h.rnd_l_base_except_entrance_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 1.82, 1.82, 0.0)
基礎断熱住宅 : (10.61, 2.97, 9.24, 4.79)


In [170]:
show_result(lambda h: h.rnd_l_base_except_entrance_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 3.64
基礎断熱住宅 : 0.0


## その他の土間床等の外周部の長さ(m<sup>2</sup>)

In [171]:
for h in hs:
    h.rnd_l_base_other_outside = round2_direction((
        round2(h.l_prm_outside[0]) - round2(h.l_base_entrance_outside[0]) - round2(h.l_base_bath_outside[0]),
        round2(h.l_prm_outside[1]) - round2(h.l_base_entrance_outside[1]) - round2(h.l_base_bath_outside[1]),
        round2(h.l_prm_outside[2]) - round2(h.l_base_entrance_outside[2]) - round2(h.l_base_bath_outside[2]),
        round2(h.l_prm_outside[3]) - round2(h.l_base_entrance_outside[3]) - round2(h.l_base_bath_outside[3])
    ))
    h.rnd_l_base_other_inside = round2(round2(h.l_prm_inside) - round2(h.l_base_entrance_inside) - round2(h.l_base_bath_inside))

In [172]:
show_result(lambda h: h.rnd_l_base_other_outside)

床断熱住宅（外皮に接する浴室床なし） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室床断熱） : (0.0, 0.0, 0.0, 0.0)
床断熱住宅（浴室基礎断熱） : (0.0, 0.0, 0.0, 0.0)
基礎断熱住宅 : (10.61, 1.15, 7.42, 4.79)


In [173]:
show_result(lambda h: h.rnd_l_base_other_inside)

床断熱住宅（外皮に接する浴室床なし） : 0.0
床断熱住宅（浴室床断熱） : 0.0
床断熱住宅（浴室基礎断熱） : 0.0
基礎断熱住宅 : 0.0


# テスト

In [174]:
def check_result(e,r):
    if abs( e - r ) < 0.000001:
        return ('EXPECTED = ' + str(e) + ' / RESULT = ' +  str(r) + ' / ' + ('.....OK'), True)
    else:
        return ('EXPECTED = ' + str(e) + ' / RESULT = ' +  str(r) + ' / ' + ('.....NG'), False)

In [175]:
def check_tuple_result(e,r):
    
    messages = []
    results = True
    
    message, result = check_result(e=e[0], r=r[0])
    if not result:
        messages.append('（南西）: ' + message)
        results = result
        
    message, result = check_result(e=e[1], r=r[1])
    if not result:
        messages.append('（北西）: ' + message)
        results = result
        
    message, result = check_result(e=e[2], r=r[2])
    if not result:
        messages.append('（北東）: ' + message)
        results = result

    message, result = check_result(e=e[3], r=r[3])
    if not result:
        messages.append('（南東）: ' + message)
        results = result

    return messages, results

In [176]:
testcase = [
    ['外皮の部位の面積の合計', '', lambda h: h.rnd_a_env,
     262.46, 262.46, 266.10, 275.69],
    ['床面積の合計', '', lambda h: h.rnd_a_f_total,
     90.00, 90.00, 90.00, 90.00],
    ['屋根又は天井の面積', '', lambda h: h.rnd_a_roof,
     50.85, 50.85, 50.85, 50.85],
    ['壁の面積', '', lambda h: h.rnd_a_wall,
     (30.47, 22.37, 47.92, 22.28),
     (30.47, 22.37, 47.92, 22.28),
     (30.47, 22.37, 47.92, 22.28),
     (30.47, 22.37, 47.92, 22.28)],
    ['ドアの面積', '', lambda h: h.rnd_a_door,
     (0.00, 1.89, 1.62, 0.00),
     (0.00, 1.89, 1.62, 0.00),
     (0.00, 1.89, 1.62, 0.00),
     (0.00, 1.89, 1.62, 0.00)],
    ['窓の面積', '', lambda h: h.rnd_a_wnd,
     (22.69, 2.38, 3.63, 4.37),
     (22.69, 2.38, 3.63, 4.37),
     (22.69, 2.38, 3.63, 4.37),
     (22.69, 2.38, 3.63, 4.37)],
    ['浴室の床断熱した床の面積', '（2019/10/15 に追加。評価方法には2018時点で反映されている。）', lambda h: h.rnd_a_if_bath,
     3.31, 3.31, 0.00, 0.00],
    ['床断熱した床の面積', '', lambda h: h.rnd_a_if,
     45.05, 45.05, 45.05, 0.00],
    ['玄関等の基礎の面積（外気側）', '(2017.07より使用。)', lambda h: h.rnd_a_base_entrance_outside,
     (0.00, 0.33, 0.25, 0.00), 
     (0.00, 0.33, 0.25, 0.00), 
     (0.00, 0.33, 0.25, 0.00), 
     (0.00, 0.33, 0.25, 0.00)],
    ['玄関等の基礎の面積（床下側）', '(2017.07より使用。)', lambda h: h.rnd_a_base_entrance_inside,
     0.57, 0.57, 0.57, 0.00],
    ['浴室の基礎の面積（外気側）', '(参考値。)', lambda h: h.rnd_a_base_bath_outside,
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.91, 0.91, 0.00),
     (0.00, 0.91, 0.91, 0.00)],
    ['浴室の基礎の面積（床下側）', '(参考値。)', lambda h: h.rnd_a_base_bath_inside,
     0.00, 0.00, 1.82, 0.00],
    ['その他の基礎の面積（外気側）', '(参考値。)', lambda h: h.rnd_a_base_other_outside,
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     (5.30, 0.57, 3.71, 2.40)],
    ['その他の基礎の面積（床下側）', '(参考値。)', lambda h: h.rnd_a_base_other_inside,
     0.00, 0.00, 0.00, 0.00],    
    ['玄関等の土間床等の外周部の長さ（外気側）', '(2017.07より使用。)', lambda h: h.rnd_l_base_entrance_outside,
     (0.00, 1.82, 1.37, 0.00),
     (0.00, 1.82, 1.37, 0.00),
     (0.00, 1.82, 1.37, 0.00),
     (0.00, 1.82, 1.37, 0.00)],
    ['玄関等の土間床等の外周部の長さ（床下側）', '(2017.07より使用。)', lambda h: h.rnd_l_base_entrance_inside,
     3.19, 3.19, 3.19, 0.00],
    ['浴室の土間床等の外周部の長さ（外気側）', '(参考値。)', lambda h: h.rnd_l_base_bath_outside,
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 1.82, 1.82, 0.00),
     (0.00, 1.82, 1.82, 0.00)],
    ['浴室の土間床等の外周部の長さ（床下側）', '(参考値。)', lambda h: h.rnd_l_base_bath_inside,
     0.00, 0.00, 3.64, 0.00],
    ['その他の土間床等の外周部の長さ（外気側）', '(参考値。)', lambda h: h.rnd_l_base_other_outside,
     ( 0.00, 0.00, 0.00, 0.00),
     ( 0.00, 0.00, 0.00, 0.00),
     ( 0.00, 0.00, 0.00, 0.00),
     (10.61, 1.15, 7.42, 4.79)],
    ['その他の土間床等の外周部の長さ（床下側）', '(参考値。)', lambda h: h.rnd_l_base_other_inside,
     0.00, 0.00, 0.00, 0.00],
    
    ['土間床等の面積の合計', '(参考)', lambda h: h.rnd_a_ef_total,
     2.48, 2.48, 5.80, 50.85],
    ['浴室の土間床等の面積', '(2018.04から使用。)', lambda h: h.rnd_a_ef_bath,
     0.00, 0.00, 3.31, 3.31],
    ['基礎の面積の合計（外気側）', '(2017.07まで使用。現在は使用していない。)', lambda h: h.rnd_a_base_total_outside,      
     (0.00, 0.33, 0.25, 0.00),
     (0.00, 0.33, 0.25, 0.00),
     (0.00, 1.24, 1.16, 0.00),
     (5.30, 1.81, 4.87, 2.40)],
    ['基礎の面積の合計（床下側）', '(2017.07まで使用。現在は使用していない。)', lambda h: h.rnd_a_base_total_inside,
     0.58, 0.58, 2.39, 0.00],
    ['玄関等を除く基礎の面積（外気側）', '(2017.07より使用。)', lambda h: h.rnd_a_base_except_entrance_outside,
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.91, 0.91, 0.00),
     (5.30, 1.48, 4.62, 2.40)],
    ['玄関等を除く基礎の面積（床下側）', '(2017.07より使用。)', lambda h: h.rnd_a_base_except_entrance_inside,
     0.00, 0.00, 1.82, 0.00],
    ['土間床周長（外気側）', '(2017.07まで使用。現在は使用していない。)', lambda h: h.rnd_l_prm_outside,  
     ( 0.00, 1.82,  1.37, 0.00),
     ( 0.00, 1.82,  1.37, 0.00),
     ( 0.00, 3.64,  3.19, 0.00),
     (10.61, 4.79, 10.61, 4.79)],
    ['土間床周長（床下側）', '(2017.07まで使用。現在は使用していない。)', lambda h: h.rnd_l_prm_inside,
     3.19, 3.19, 6.83, 0.00],
    ['玄関等を除く土間床等の外周部の長さ（外気側）', '(2017.07より使用。)', lambda h: h.rnd_l_base_except_entrance_outside,
     (0.00, 0.00, 0.00, 0.00),
     (0.00, 0.00, 0.00, 0.00),
     ( 0.00, 1.82, 1.82, 0.00),
     (10.61, 2.97, 9.24, 4.79)],
    ['玄関等を除く土間床等の外周部の長さ（床下側）', '(2017.07より使用。)', lambda h: h.rnd_l_base_except_entrance_inside,
     0.00, 0.00, 3.64, 0.00],

]

In [177]:
for index, row in enumerate(testcase):
    
    name, remark, func = row[0], row[1], row[2]

    if type(row[3]) == str or type(row[3]) == float:
        for i, h in enumerate(hs):
            message, result = check_result(e=row[i+3], r=func(h))
            if result:
                pass
            else:
                print(name + ' ' + remark + ': ' + h.name + ' : ' + message)
    elif type(row[3]) == tuple and len(row[3]) == 4:
        for i, h in enumerate(hs):
            messages, results = check_tuple_result(e=row[i+3], r=func(h))
            if not results:
                [print(name + ' ' + remark + ': ' + h.name + m) for m in messages]
    else:
        print ( '  Format Error' )

基礎の面積の合計（床下側） (2017.07まで使用。現在は使用していない。): 床断熱住宅（外皮に接する浴室床なし） : EXPECTED = 0.58 / RESULT = 0.57 / .....NG
基礎の面積の合計（床下側） (2017.07まで使用。現在は使用していない。): 床断熱住宅（浴室床断熱） : EXPECTED = 0.58 / RESULT = 0.57 / .....NG


In [178]:
wb = xlrd.open_workbook('publicated_value.xlsx')
sheet = wb.sheet_by_name('test')
value = [sheet.row_values(row) for row in range(sheet.nrows)]